In [ ]:
import os
import pandas as pd
import numpy as np
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.metrics import classification_report

In [ ]:
paths = 'content/coinbase_btc_usd/coinbase/btc_usd/l2_snapshots/100ms/'

In [ ]:
register = {}

In [ ]:
l2_snapshot = pd.DataFrame()

In [ ]:
day = pd.DataFrame()

In [ ]:
count = 0
day = pd.DataFrame()
i = 0
for x in os.listdir(paths):
    path = paths+x
    temp = pd.read_parquet(path)
    if count%24 == 0:
        day = pd.read_parquet(path)
    else:
        day = pd.concat([day,temp])
  
    i+=1
    count +=1

    if count%24 == 0:
        flag = not register
        day = day.dropna()
        print(day.shape)
        result = []
        for cols in day.columns:
            values = day[cols].to_numpy()
            mean = np.mean(values)
            std = np.std(values)
            result.append([cols,[mean,std]])
        register[count/24] = result
        if not flag:
            for l in range(200):
                prev_stat = register[(count/24)-1]
                col_name = prev_stat[l][0]
                col_mean = prev_stat[l][1][0]
                col_std = prev_stat[l][1][1]
                values = day[col_name].to_numpy()
                values = (values - col_mean)/col_std
                day[col_name] = values
            l2_snapshot = pd.concat([day,l2_snapshot])
        del day
  
    if i>240:
        break

In [ ]:
l2_snapshot_ = l2_snapshot[['b1','b2','b3','b4','b5','b6','b7','b8', 'b9', 'b10', 'a1','a2','a3','a4','a5','a6','a7','a8', 'a9', 'a10', 'bq1','bq2','bq3','bq4','bq5','bq6','bq7','bq8', 'bq9', 'bq10', 'aq1','aq2','aq3','aq4','aq5','aq6','aq7','aq8', 'aq9', 'aq10']]

In [ ]:
def generate_features_labels(df,ask, bid,k=20, alpha = 10e-5):
    df_ = df.copy()
    df_['mid_price'] = (df_[ask].to_numpy()+df_[bid].to_numpy())/2
    df_['target'] = 1
    index = df_.columns.get_loc('mid_price')
    target_index = df_.columns.get_loc('target')
    shape = df_.shape[0]
    y = df_['target'].to_numpy()
    for i in tqdm(range(k,shape-k)):
        if i==k:
            m_b = np.mean(df_.iloc[(i-k):i, index].to_numpy())
            m_a = np.mean(df_.iloc[i+1:(i+k+1), index].to_numpy())
            val = df_.iloc[i-k,index]
            valB = df_.iloc[i+1, index]
        else:
            curr_val = df_.iloc[i-1,index]
            curr_valB = df_.iloc[i+k, index]
            m_b = (m_b*k+(curr_val)-(val))/k
            m_a = (m_a*k+(curr_valB - valB))/k
            val = df_.iloc[(i-k),index]
            valB = df_.iloc[(i+1), index]

        if (m_b > m_a*(1+alpha)):
            y[i] = 2
        if (m_b < m_a*(1-alpha)):
            y[i] = 0

    y = y[k:shape-k]
    X = df.iloc[k:shape-k,:].to_numpy()
    return X,y

In [ ]:
# GridSearch
min_ = float('inf')
minI = 0
for i in np.logspace(-5,-20,20, endpoint = True):
    X,y = generate_features_labels(l2_snapshot, ask='a1', bid = 'b1', alpha=i)
    y_ = pd.Series(y)
    temp = y_.value_counts()
    a,b,c = temp[0], temp[1], temp[2]
    sum_ = a+b+c
    l2 = ((((1/3-(a/sum_)))**2 + ((1/3-(b/sum_)))**2 + ((1/3-(c/sum_)))**2)/3)**2
    if l2 < min_:
        min_ = l2
        minI = i
    print(min_, minI)

In [ ]:
print(minI, min_)

In [ ]:
X,y = generate_features_labels(l2_snapshot_, ask='a1', bid = 'b1', alpha=7.847599703514623e-13)

In [ ]:
l2_snapshot_['mid_price'] = (l2_snapshot_['b1']+l2_snapshot_['a1'])/2

In [ ]:
plt.figure(figsize = (20,15))
plt.plot(np.arange(1,l2_snapshot_.shape[0]+1), l2_snapshot_['mid_price'])

In [ ]:
y = y.astype('int')

In [ ]:
sns.countplot(x = y)

In [ ]:
T = 100
D = 40
N = len(X) - T

In [ ]:
trainPart = 3176026

In [ ]:
X_train = np.zeros((trainPart, T, D), dtype = 'float32')
y_train = np.zeros(trainPart, dtype='float32')

#Preparing the time series data using timestep of 100 and no of features = 200
for t in range(trainPart):
    X_train[t, :, :] = X[t:t+T]
    y_train[t] = y[t+T] 


In [ ]:
print('X_train - Before: {} GB'.format(X_train.nbytes/1024**3), X_train.dtype)

In [ ]:
print('y_train - Before: {} GB'.format(y_train.nbytes/1024**3), y_train.dtype)

In [ ]:
X_test = np.zeros((N - trainPart, T, D), dtype = 'float32')
y_test = np.zeros(N - trainPart, dtype='float32')

#Preparing the time series data using timestep of 100 and no of features = 40
for k in range(N - trainPart):
    t = k + trainPart
    X_test[k, :, :] = X[t:t+T]
    y_test[k] = y[t+T]

print('X_test - Before: {} GB'.format(X_test.nbytes/1024**3), X_test.dtype)
print('y_test - Before: {} GB'.format(y_test.nbytes/1024**3), y_test.dtype)

In [ ]:
import tensorflow as tf
print(tf.__version__)

In [ ]:
!pip install tensorflow-addons

In [ ]:
import tensorflow_addons as tfa

In [ ]:
from tensorflow.keras.layers import Input, Conv1D, Conv2D, Flatten, MaxPooling1D, MaxPooling2D, Dense, LeakyReLU, Bidirectional, LSTM, Add, Activation, GlobalAveragePooling1D, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.compat.v1.keras.layers import CuDNNLSTM

In [ ]:
from tensorflow import keras
from functools import partial

In [ ]:
## Resnet of Conv1D
DefaultConv1D = partial(keras.layers.Conv1D, kernel_size=4, strides=1,
                        padding="SAME", use_bias=False)

class ResidualUnit(keras.layers.Layer):
    def __init__(self, filters, strides=1, activation="relu", **kwargs):
        super().__init__(**kwargs)
        self.activation = keras.activations.get(activation)
        self.main_layers = [
            DefaultConv1D(filters, strides=strides),
            keras.layers.BatchNormalization(),
            self.activation,
            DefaultConv1D(filters),
            keras.layers.BatchNormalization()]
        self.skip_layers = []
        if strides > 1:
            self.skip_layers = [
                DefaultConv1D(filters, kernel_size=1, strides=strides),
                keras.layers.BatchNormalization()]

    def call(self, inputs):
        Z = inputs
        for layer in self.main_layers:
            Z = layer(Z)
        skip_Z = inputs
        for layer in self.skip_layers:
            skip_Z = layer(skip_Z)
        return self.activation(Z + skip_Z)

In [ ]:
strategy = tf.distribute.MirroredStrategy()
with strategy.scope():
    inputs = keras.layers.Input(shape = (100,40))
    x = keras.layers.Conv1D(filters=16, kernel_size=4 ,activation='relu', padding='SAME')(inputs)
    x = keras.layers.MaxPooling1D(2)(x)
    
    prev_filters = 16
    for filters in [16]*3+[32]*2:
        if prev_filters != filters:
            stride = 2
        else:
            stride = 1
        x = ResidualUnit(filters, stride)(x)
        prev_filters = filters

    x = keras.layers.GlobalAveragePooling1D()(x)
    outputs = keras.layers.Dense(3, activation='softmax')(x)

    res_net_model = keras.models.Model(inputs, outputs)
    res_net_model.compile(optimizer='adam',
               loss='sparse_categorical_crossentropy',
                metrics = ['accuracy'])

In [ ]:
# strategy = tf.distribute.MirroredStrategy()
# with strategy.scope():
#     i = Input(shape=X_train[0].shape)
    
#     x = Conv2D(16, (4,D),kernel_initializer="he_normal")(i)
#     x = LeakyReLU(alpha=0.01)(x)
#     x = tf.keras.layers.BatchNormalization()(x)
    
#     x = tf.keras.layers.Reshape(target_shape=(T-3,16))(x)
    
#     x = Conv1D(16, 4, kernel_initializer="he_normal")(x)
#     x = LeakyReLU(alpha=0.01)(x)
#     x = tf.keras.layers.BatchNormalization()(x)
#     x = MaxPooling1D(2)(x)
    
#     x = Conv1D(32, 3, activation=LeakyReLU(alpha=0.01), kernel_initializer="he_normal")(x)
#     x = LeakyReLU(alpha=0.01)(x)
#     x = tf.keras.layers.BatchNormalization()(x)
#     x = MaxPooling1D(2)(x)
    
#     x = Bidirectional(LSTM(64, return_sequences = False))(x)
    
#     x = Dense(32 ,kernel_initializer="he_normal")(x)
#     x = LeakyReLU(alpha=0.01)(x)
#     x = tf.keras.layers.BatchNormalization()(x)
    
#     x = Dense(32, kernel_initializer="he_normal")(x)
#     x = LeakyReLU(alpha=0.01)(x)
#     x = tf.keras.layers.BatchNormalization()(x)
    
#     x = Dense(3, activation='softmax', kernel_initializer="he_normal")(x)

#     model = Model(i,x)
#     model.compile(optimizer='adam',
#               loss='sparse_categorical_crossentropy',
#               metrics=['accuracy'])
    

In [ ]:
EPOCHS = 2
with tf.device('/CPU:0'):
    r = res_net_model.fit(
        X_train,y_train,
        epochs=EPOCHS,
        batch_size=128
    )

In [ ]:
res_net_model.evaluate(X_test, y_test)

In [ ]:
y_pred = res_net_model.predict(X_test)

In [ ]:
sns.lineplot(x=range(1,EPOCHS+1),y=r.history['loss'])
#sns.lineplot(x=range(1,EPOCHS+1),y=r.history['val_loss'])
plt.title('Model Cross Entropy Loss')
plt.ylabel('Cross Entropy Loss')
plt.xlabel('Epochs')
plt.legend(['Training'], loc='upper left')

In [ ]:
y_pred = y_pred.argmax(axis=1)

In [ ]:
with strategy.scope():
    y_pred = model.predict(X_test).argmax(axis=1)

In [ ]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_pred=y_pred, y_true=y_test)
    
cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

ax=sns.heatmap(cm, annot=True, xticklabels=[0,1,2], yticklabels=[0,1,2], cmap='Blues')
ax.set_ylim(3.0, 0)
plt.show()

In [ ]:
print(classification_report(y_test,y_pred))